In [1]:
import pandas as pd
import ast
import json
import os

with open('config.json', 'r') as f:
    config = json.load(f)

path = config['working_dir']

output_dir = os.path.join(path,'output_bert_no_dim_new_new2') # Directory di output output_embedding_openai
print('output_dir: ',output_dir)

prefix = 'BERT' # openai
print('output_file prefix:',prefix)

os.makedirs(output_dir, exist_ok=True)  # Crea la directory di output se non esiste

output_dir:  /Users/francescamartinucci/tesi/output_bert_no_dim_new_new2
output_file prefix: BERT


In [3]:
def process_embeddings(input_text, input_seq, model_name):
    """ 
    """
    print(model_name)
    
    # # Unisce verticalmente txt e seq
    # df_t = pd.read_csv(input_seq, sep="\t", usecols=['name', 'embedding'])
    # df_t.set_index('name', inplace=True)
    # df_d = pd.read_csv(input_text, sep="\t", usecols=['name', 'embedding'])
    # df_d.set_index('name', inplace=True)
    
    # df_b = pd.concat([df_t, df_d], axis=0)
    df_b = pd.read_csv(input_seq, sep="\t", usecols=['name', 'embedding'])
    df_b.set_index('name', inplace=True)
    print(len(df_b['embedding'].apply(ast.literal_eval).iloc[0]))

    # Legge gli embeddings Grape
    df_fo = pd.read_csv(os.path.join(path, f"prediction_output/{model_name}_dim100_0.tsv"), sep="\t",usecols=['name', 'embedding'])
    df_fo.set_index('name', inplace=True)
    print(len(df_fo['embedding'].apply(ast.literal_eval).iloc[0]))

    # Join dei due dataframe in base all'indice
    df = pd.merge(df_b, df_fo, left_index=True, right_index=True, suffixes=('_l', '_r'))  # default inner join

    # Applica la funzione per convertire la stringa in array numerico e per concatenare gli array
    df['embedding'] = df.apply(lambda row: ast.literal_eval(row['embedding_l']) + ast.literal_eval(row['embedding_r']), axis=1)
    df.drop(columns=['embedding_l', 'embedding_r'], inplace=True)

    emb_length = len(df['embedding'].iloc[0])
    print(emb_length)

    # Espandi la colonna 'embedding' in un nuovo DataFrame
    exploded_df = pd.DataFrame(df['embedding'].tolist(), index=df.index)

    # Concatenazione lungo le colonne (axis=1)
    df_final = pd.concat([df, exploded_df], axis=1)

    df_final = df_final.drop(columns=['embedding'])
    df_final.to_csv(os.path.join(output_dir, f'{prefix}_{model_name}100_0.csv'))


In [4]:
input_text = os.path.join(output_dir, "embedded_text_nan_filled.tsv") # non usato
input_seq = os.path.join(output_dir, "BERT_embeddings.tsv")

process_embeddings(input_text,input_seq,'line')
process_embeddings(input_text,input_seq,'node2vec')
process_embeddings(input_text,input_seq,'transe')

line
768
100
868
node2vec
768
100
868
transe
768
100
868
